# Geospatial API Project
Silver Sloths

In [1]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json
from io import StringIO

In [2]:
# Find all aggravated burglary incidents that were reported from January 1, 2021 through June 30, 2021.
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.csv'
response = requests.get(endpoint, {'$limit': 1000000})
burglary = pd.read_csv(StringIO(response.text))
burglary

,primary_key,incident_number,report_type,report_type_description,incident_status_code,incident_status_description,investigation_status,incident_occurred,incident_reported,incident_location,...,weapon_description,victim_number,domestic_related,victim_type,victim_description,victim_gender,victim_race,victim_ethnicity,victim_county_resident,mapped_location
0,2021260418_11,2021260418,W,WITNESS,O,OPEN,Open,2021-04-14T17:00:00.000,2021-04-21T16:40:00.000,HILLSIDE AVE,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.78 36.14)
1,20210513835_11,20210513835,D,DISPATCHED,U,UNFOUNDED,Closed,2021-09-30T21:23:00.000,2021-09-30T21:47:00.000,105 105,...,NONE,1,True,I,INDIVIDUAL (18 AND OVER),M,U,Non-Hispanic,RESIDENT,POINT (-86.717 36.263)
2,20210513823_11,20210513823,D,DISPATCHED,U,UNFOUNDED,Closed,2021-09-30T19:45:00.000,2021-09-30T22:43:00.000,1333 1333,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,B,Non-Hispanic,NON RESIDENT,POINT (-86.772 36.196)
3,20210513703_11,20210513703,D,DISPATCHED,U,UNFOUNDED,Closed,2021-09-30T19:38:00.000,2021-09-30T21:39:00.000,109 109,...,NONE,1,True,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,POINT (-86.712 36.263)
4,20210513650_11,20210513650,D,DISPATCHED,O,OPEN,Open,2021-09-30T19:22:00.000,2021-09-30T19:57:00.000,OLD MATTHEWS RD,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,POINT (-86.79 36.22)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884525,20100814105_11,20100814105,D,DISPATCHED,O,OPEN,Open,2014-09-01T15:43:00.000,2014-09-02T10:30:00.000,VAILVIEW DR,...,CLUB,1,False,G,GOVERNMENT,NaN,NaN,NaN,NaN,POINT (-86.77 36.24)
884526,1960000002_11,1960000002,D,DISPATCHED,O,OPEN,Open,2015-02-03T15:30:00.000,2015-02-03T16:49:00.000,FESSLERS LN,...,PERSONAL (HANDS),1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,NON RESIDENT,POINT (-86.74 36.15)
884527,19000000985_11,19000000985,D,DISPATCHED,O,OPEN,Open,2021-08-23T08:00:00.000,2021-08-23T12:16:00.000,DICKERSON PIKE,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),U,U,Unknown,NON RESIDENT,POINT (-86.76 36.23)
884528,19000000981_11,19000000981,T,NaN,O,OPEN,Open,2018-09-13T16:00:00.000,2018-09-13T16:56:00.000,MASSMAN DR,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,POINT (-86.71 36.15)
